<a href="https://colab.research.google.com/github/zarifemrah/crewAI/blob/main/CrewAI_developer_tester_TSP_ORtools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Setup and Tool Definition

First, we set up the environment and define a custom CodeExecutionTool. This tool is essential as it gives our testing agent the ability to run the Python code provided by the developer agent and capture any output or errors.

In [ ]:
!pip install crewai crewai_tools ortools

In [ ]:
import os
from google.colab import userdata
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
import io
from contextlib import redirect_stdout, redirect_stderr

# Set up your OpenAI API key from Colab Secrets
# 1. Click the key icon (🔑) on the left sidebar
# 2. Add a new secret with the name "OPENAI_API_KEY" and your key as the value
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Define the custom tool for code execution ⚙️
class CodeExecutionTool(BaseTool):
    name: str = "Code Execution Tool"
    description: str = "Executes Python code and returns the output or error. Input should be a string of Python code."

    def _run(self, code: str) -> str:
        # Use io.StringIO to capture print outputs
        output_buffer = io.StringIO()
        error_buffer = io.StringIO()

        try:
            # Redirect stdout and stderr to capture the execution results
            with redirect_stdout(output_buffer), redirect_stderr(error_buffer):
                # Execute the code
                exec(code, {})

            # Get the captured output and errors
            output = output_buffer.getvalue().strip()
            error = error_buffer.getvalue().strip()

            if error:
                return f"Execution failed with error: {error}"
            elif output:
                return f"Execution successful with output: {output}"
            else:
                return "Execution successful with no output."

        except Exception as e:
            return f"An exception occurred during execution: {str(e)}"

# Instantiate the tool for the tester agent
code_executor = CodeExecutionTool()

## 2. Define Agents and Tasks

Next, we define the two agents: the developer and the tester. The tester agent will be equipped with the CodeExecutionTool we just created. The tasks are designed so that the testing task uses the output of the coding task.

In [ ]:
# New user-defined algorithm for TSP using an optimization package
algorithm_description = (
    "Create a Python function that solves the Traveling Salesman Problem (TSP) using the "
    "Google OR-Tools library. "
    "\n\nThe function should be named `solve_tsp_ortools` and take one argument:"
    "\n1. `distance_matrix`: A 2D list representing the distances between cities."
    "\n\nThe implementation must use `ortools.constraint_solver.pywrapcp` to create a routing model, "
    "find the optimal solution, and return a tuple containing:"
    "\n1. `path`: A list of city indices for the optimal tour, starting and ending at city 0."
    "\n2. `distance`: The total distance of the optimal tour."
    "\n\nInclude a test case that calls the function with a sample distance matrix and prints the resulting path and distance."
)

# Define the Developer Agent 🧑‍💻
developer = Agent(
    role='Senior Python Developer',
    goal=f'Write clean, efficient, and error-free Python code for the following task: {algorithm_description}.',
    backstory=(
        "You are an experienced Python developer with a knack for writing robust and elegant code. "
        "You receive algorithm descriptions and turn them into functional Python scripts. "
        "When you receive feedback or errors, you diligently debug and refactor your code to meet the requirements."
    ),
    verbose=True,
    allow_delegation=False
)

# Define the Tester Agent 🧪
tester = Agent(
    role='Software Quality Assurance Engineer',
    goal='Test the Python code provided by the developer. Identify any bugs, runtime errors, or logical flaws. Provide clear, concise feedback for fixes.',
    backstory=(
        "You are a meticulous QA engineer with a sharp eye for detail. Your job is to take Python scripts, "
        "run them using your execution tool, and rigorously test them. If you find any issue, "
        "you report it back to the developer with the error message and suggestions for how to fix it."
    ),
    tools=[code_executor], # Assign the execution tool to the tester
    verbose=True,
    allow_delegation=True # IMPORTANT: Allows the tester to delegate back to the developer
)

# Define the Tasks
coding_task = Task(
    description=f"Write a Python script that implements the following: {algorithm_description}. Your final output must be ONLY the complete, runnable Python code.",
    expected_output="A single block of clean, executable Python code that defines the `solve_tsp_ortools` function, imports necessary OR-Tools modules, and includes a test case.",
    agent=developer
)

testing_task = Task(
    description=(
        "You are responsible for testing the Python code for the Traveling Salesman Problem that uses the OR-Tools library. "
        "Use your 'Code Execution Tool' to run the script."
        "\n\nThe test case within the script should use the same 4x4 distance matrix as our previous tasks. "
        "The optimal solution for that matrix has a total distance of 80."
        "\n\nYour task is to verify that the script's output contains the line 'Total Distance: 80'. "
        "The path may be '[0, 1, 3, 2, 0]' or '[0, 2, 3, 1, 0]', but the distance is the critical part."
        "\n\nIf the code runs and the distance is correct, your final answer is the code itself with a confirmation. "
        "If there is any error or the distance is not 80, delegate back to the developer with a report."
    ),
    expected_output="The final, correct Python code along with the confirmation 'Code is tested and works perfectly with an optimal distance of 80.' OR a detailed error/discrepancy report for the developer.",
    agent=tester,
    context=[coding_task] # The testing task depends on the coding task
)

## 3. Create the Crew and Kick Off the Task

Finally, we assemble the agents and tasks into a Crew. We use the sequential process, and crewAI's internal delegation mechanism will handle the back-and-forth between the agents until the testing_task is completed successfully.

In [ ]:
# Create the crew with a sequential process
code_crew = Crew(
    agents=[developer, tester],
    tasks=[coding_task, testing_task],
    process=Process.sequential,
    verbose=True # Set to 2 for detailed logging of the process
)

# Kick off the crew's work 🚀
result = code_crew.kickoff()

# Print the final result
print("\n\n########################")
print("## Crew Work Complete!")
print("########################\n")
print("Final Result:")
print(result)